## Import libraries

In [49]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap
from fiona.crs import from_epsg

## Data addresses

In [5]:
stations_request = "http://api.gios.gov.pl/pjp-api/rest/station/findAll"
sensors_request = "http://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId}"
data_request = "http://api.gios.gov.pl/pjp-api/rest/data/getData/{sensorId}"
aqindex_request = "http://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/{stationId}"

## Get the data

In [6]:
stations = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/findAll").json()
print("Number of available stations: ", len(stations))

Number of available stations:  151


In [7]:
stations_dict = {}
station_ids = []
station_lats = []
station_lons = []
station_geometries = []

for station in stations:
    
    station_ids.append(station["id"])
    station_lats.append(float(station["gegrLat"]))
    station_lons.append(float(station["gegrLon"]))
    station_geometries.append(Point(float(station["gegrLon"]), float(station["gegrLat"])))
    
stations_dict["station_id"] = station_ids
stations_dict["lat"] = station_lats
stations_dict["lon"] = station_lons
stations_dict["station_id"] = station_ids
stations_dict["geometry"] = station_geometries

In [8]:
stations_df = gpd.GeoDataFrame(stations_dict)
stations_df.crs = from_epsg(4326)
stations_df.head()

,station_id,lat,lon,geometry
0,114,51.115933,17.141125,POINT (17.141125 51.115933)
1,129,51.086225,17.012689,POINT (17.012689 51.086225)
2,52,51.204503,16.180513,POINT (16.180513 51.204503)
3,109,50.768729,16.269677,POINT (16.269677 50.768729)
4,38,50.433493,16.653660,POINT (16.65366 50.433493)


## Plot stations on map

In [40]:
stations_map = folium.Map([52, 19], zoom_start=6, tiles='Stamen Terrain')

points = folium.features.GeoJson(stations_df.to_json())

stations_map.add_child(points)
stations_map.add_child(HeatMap([[row["lat"], row["lon"]] for name, row in stations_df.iterrows()]))
stations_map

## Check available readings for stations

In [51]:
for station in stations:
    
    print("Available readings for station_id: ", station["id"])
    
    station_id = station["id"]
    sensors = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    for sensor in sensors:
        print(sensor["id"], sensor["param"])
    
    print()

Available readings for station_id:  114
642 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
644 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}

Available readings for station_id:  129
737 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
740 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
744 {'paramName': 'pył zawieszony PM2.5', 'paramFormula': 'PM2.5', 'paramCode': 'PM2.5', 'idParam': 69}

Available readings for station_id:  52
285 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
14397 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
282 {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}
291 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
293 {'paramName': 'ozon', 'paramFormula': 'O

16228 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
16250 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
16249 {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}
16231 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}

Available readings for station_id:  291
2035 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
2031 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
2033 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
2039 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for station_id:  319
2219 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
2221 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'i

6062 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
6067 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
6065 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
6071 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for station_id:  944
6076 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
6085 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
6074 {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}
6081 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
6083 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
6087 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for 

5564 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
5573 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
5568 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
5570 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
5579 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for station_id:  834
5457 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
5466 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
5454 {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}
5461 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
5463 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
5470 {'paramName': 'dwutlenek siarki',

2770 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
17896 {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}
2766 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
2772 {'paramName': 'pył zawieszony PM2.5', 'paramFormula': 'PM2.5', 'paramCode': 'PM2.5', 'idParam': 69}
2768 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
2774 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for station_id:  402
2783 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
2792 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
2779 {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}
2788 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
2794 {'paramNam

4755 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
4761 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
4758 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
4762 {'paramName': 'pył zawieszony PM2.5', 'paramFormula': 'PM2.5', 'paramCode': 'PM2.5', 'idParam': 69}
4760 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
4764 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for station_id:  738
4774 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
4769 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
4772 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
4777 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readi

3409 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
3407 {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}
3412 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
3414 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
3418 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for station_id:  501
3420 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
3429 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
3419 {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}
3425 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
14376 {'paramName': 'pył zawieszony PM2.5', 'paramFormula': 'PM2.5', 'paramCode': 'PM2.5', 'idParam': 69}
16765 {'paramName': 'ozon'

2589 {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}
14618 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
2594 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
2596 {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}
2601 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for station_id:  145
853 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
850 {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}
855 {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}

Available readings for station_id:  206
1405 {'paramName': 'pył zawieszony PM10', 'paramFormula': 'PM10', 'paramCode': 'PM10', 'idParam': 3}
1407 {'paramName': 'pył zawieszony PM2.5', 'paramFormula': 'PM2

In [43]:
sensors_dict = {}
stations_ids = []
sensors_ids = []
sensors_param = []

for station in stations:
    
    station_id = station["id"]
    sensors = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    for sensor in sensors:

        stations_ids.append(sensor["stationId"])
        sensors_ids.append(sensor["id"])
        sensors_param.append(sensor["param"]["paramCode"])
    
sensors_dict["station_id"] = stations_ids
sensors_dict["sensor_id"] = sensors_ids
sensors_dict["parameter"] = sensors_param\

In [44]:
sensors_df = pd.DataFrame(sensors_dict)
sensors_df.head()

,station_id,sensor_id,parameter
0,114,642,NO2
1,114,644,O3
2,129,737,CO
3,129,740,NO2
4,129,744,PM2.5


## Check data

In [48]:
for sensor_id in sensors_df['sensor_id']:

    data = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    print(data["values"][0]["value"])
    print()

33.1148

27.8406

1124.5700000000002

70.6755

None

554.361

None

1.36989

56.3642

12.9861

2.1845

355.34

None

0.00102

9.35658

51.9139

3.26515

None

31.0557

37.9756

1.89013

None

24.7481

2.4925

5.07264

54.8778

1.34492

66.8729

None

23.7635

4.88258

283.449

0.50425

34.3815



IndexError: list index out of range

In [53]:
def get_sensor_readings(row):
    
    sensor_id = row["sensor_id"]
    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    try:
        return float(data_json["values"][0]["value"])
    except:
        return np.NaN

In [54]:
sensors_df['data'] = sensors_df.apply(get_sensor_readings, axis=1)

In [55]:
sensors_df.head()

,station_id,sensor_id,parameter,data
0,114,642,NO2,33.1148
1,114,644,O3,27.8406
2,129,737,CO,1124.5700
3,129,740,NO2,70.6755
4,129,744,PM2.5,NaN


In [56]:
sensors_df.describe()

,station_id,sensor_id,data
count,605.000000,605.000000,319.000000
mean,2052.436364,7076.595041,55.558966
std,3495.499273,5771.337755,131.545903
min,38.000000,221.000000,0.000000
25%,401.000000,3068.000000,3.425664
50%,725.000000,4866.000000,11.126900
75%,877.000000,14378.000000,34.537275
max,11195.000000,19816.000000,1124.570000
